# Lab 4: Newton's Method for Nonlinear Systems

### Topics

- **Mathematics:**  multidimensional Newton's method; estimating errors when solving linear systems; condition number; numerical calculation of the Jacobian.
- **Python:** using Numpy's `np.linalg.solve` to solve linear systems; generalising an function for single-variable Newton's method to multidimensional Newton's method; testing for convergence of a sequence of vectors.

In [ ]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped
import scipy.linalg #we will need the hilbert function from here

%matplotlib widget
import matplotlib.pyplot as plt

## Numpy's function for solving linear systems

Numpy has a way to solve $A\underline{\mathbf{x}} = \underline{\mathbf{b}}$ using the `np.linalg.solve` function.  For example:
```
x = np.linalg.solve(A, b)
```
Mathematically, this is equivalent to $\underline{\mathbf{x}} = A^{-1} \underline{\mathbf{b}}$ (although Numpy doesn't actually work out the inverse matrix $A^{-1}$ as this is less efficient and has more issues with round-off error).

In this lab we'll use Numpy's function.

## Multidimensional Newton's method

We want to find a solution to the following nonlinear system of equations
\begin{align*}
  x_2 &= \ln x_1, \\
  x_2 &= \frac{1}{25}(x_1^2 +2x_1-24).
\end{align*}
First, rearrange this system into the form $\underline{\mathbf{f}}(\underline{\mathbf{x}}) = 0$ on paper, and write down the Jacobian matrix $J$ associated with the system.

Define a function `f` that takes in a vector `x` and returns the value of $\underline{\mathbf{f}}$ as a **1-dimensional** Numpy array, and another function `J` that takes in a vector `x` and returns the value of $J$, as a Numpy array:

In [3]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped
import scipy.linalg #we will need the hilbert function from here

%matplotlib widget
import matplotlib.pyplot as plt

def f(x):
    return(np.array([x[1] - (1/25) * (x[0]**2 - 2*x[0] + 24) , x[1] - np.log(x[0])]))
    
def J(x):
    return(np.array([[-1/x[0], 1],[1/25*(-2*x[0]-2),1]]))

x= [4,2]

print(J(x))

[[-0.25  1.  ]
 [-0.4   1.  ]]


Now look at your Newton's method function `newt` from Lab 2.  You should have a version that tests for convergence to some tolerance `tol` which begins
```
def newt(x, num_iter, func, deriv, tol=1e-4):
```
where `x` is the initial approximation to the solution, `num_iter` is the maximum number of iterations to carry out, `func` and `deriv` are the function to solve with and its derivative, and `tol` is the desired tolerance.

You are going to use this function as a template for doing multidimensional Newton's method.  Copy it into the cell below, rename it `multinewt`, and rename the `deriv` parameter to `jacobian`.  Now, modify the function so that it solves a **system** of equations, rather than just a single equation.  Remember, the key difference is that the variables $\underline{\mathbf{x}}$ and $\underline{\mathbf{f}}$ are now vectors and, instead of the derivative $\frac{\mathrm{d}f}{\mathrm{d}x}$, we now have a Jacobian matrix $J$.  The parts of the function that you need to modify are:
- `f` will now be the **vector** function you are trying to find the zeros of.
- Instead of `dfdx = ...`, you now need to define `J` to be the results of `jacobian(x)`.
- The **change** $\Delta\underline{\mathbf{x}}$ in $\underline{\mathbf{x}}$ at each iteration of Newton's method is the solution of the linear system $J\Delta\underline{\mathbf{x}} = -\underline{\mathbf{f}}$.  Use Numpy's `np.linalg.solve` to solve for `dx`.  Then update the variable `x` by adding `dx`.
- **Change** the line saving the current step to `results.append([k, *x, *f])`.  The `*` puts in the entries of the vector rather than the vector object, and will only work with 1-dimensional arrays or lists.

**Hint:** Try to make sure your function will work solving systems of any dimension (i.e. don't make it only work for 2 components).

Your function should now do the correct multidimensional Newton's method iterations.  However, you still need to modify the convergence test to account for the fact that you are using vectors instead of scalars.  So, instead of using the absolute value (`abs`), you now need to use the infinity norm (`np.linalg.norm(..., np.inf)`) to test whether

$$
  \frac{\left\|\underline{\mathbf{x}}^{\left(k+1\right)}-\underline{\mathbf{x}}^{\left(k\right)}\right\|_\infty}
       {\left\|\underline{\mathbf{x}}^{\left(k+1\right)}\right\|_\infty}
    < \mathtt{tol},
$$

where $\underline{\mathbf{x}}^{\left(k\right)}$ is the value of the vector $\underline{\mathbf{x}}$ after $k$ iterations.  Note that, by definition $\Delta \underline{\mathbf{x}} = \underline{\mathbf{x}}^{\left(k+1\right)}-\underline{\mathbf{x}}^{\left(k\right)}$.

Like your original Newton's method function, your new function should return all the iterations in a matrix called `results`.  The $k$th row of `results` should contain the values of `k` the vectors `x` and `f` after $k$ iterations.  Hopefully, the final row will contain the solution for `x`.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 1</h3>

  Use your function to find the solution(s) to the nonlinear system (there may be more than one!).  <b>Hint:</b> Use a graph to find suitable initial approximation(s) to the solution(s).
  
  Check that your answer seems reasonable.
</div>

In [10]:
import numpy as np
np.set_printoptions(linewidth=130) #set the line width so wider arrays don't get wrapped
import scipy.linalg #we will need the hilbert function from here

%matplotlib widget
import matplotlib.pyplot as plt

def f(x):
    return(np.array([x[1] - (1/25) * (x[0]**2 - 2*x[0] + 24) , x[1] - np.log(x[0])]))
    
def J(x):
    return(np.array([[-1/x[0], 1],[1/25*(-2*x[0]-2),1]]))

def multinewt(x0,func , jacobian, num_iter=100, tol=1e-4):
    results = []
    for iteration in range(num_iter):
        print(x0)
        x1 = (x0 - func(x0) / jacobian(x0)).copy()
        results.append([iteration, x1, func(x0)])
        if np.linalg.norm(x1-x0, np.inf)/np.linalg.norm(x1, np.inf) < tol:
            print(f"Root found at {x0} after {iteration + 1} iterations.")
            return np.array(results)
        x0 = x1.copy()
    
    print("Maximum number of iterations reached. No convergence.")
    return np.array(results)

print(multinewt([1,2], f, J))

[1, 2]
[[2.08 0.  ]
 [7.75 0.  ]]
[[2.08 0.  ]
 [7.75 0.  ]]


C:\Users\Tarn Holt\AppData\Local\Temp\ipykernel_5652\3511585908.py:9: RuntimeWarning: divide by zero encountered in log
  return(np.array([x[1] - (1/25) * (x[0]**2 - 2*x[0] + 24) , x[1] - np.log(x[0])]))
C:\Users\Tarn Holt\AppData\Local\Temp\ipykernel_5652\3511585908.py:12: RuntimeWarning: divide by zero encountered in divide
  return(np.array([[-1/x[0], 1],[1/25*(-2*x[0]-2),1]]))


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 2) + inhomogeneous part.

We now take a bit of a detour into ill-conditioned matrices, as covered in lectures last week.

## Errors when solving linear systems

To assess the reliability of any equation solver, you need a way of checking the accuracy of a supposed solution $\underline{\mathbf{x}}_1$ to a linear system $A\underline{\mathbf{x}}=\underline{\mathbf{b}}$.  If we write $\underline{\mathbf{x}}$ for the exact solution, then what we'd really like to know is the size of:

$$
  \begin{array}{lll}
    & \left\|\underline{\mathbf{x}}-\underline{\mathbf{x}}_1\right\| & \text{(the absolute error)}, \\
    \text{or } & \left\|\underline{\mathbf{x}}-\underline{\mathbf{x}}_1\right\|/\left\|\underline{\mathbf{x}}\right\| & \text{(the relative error)}.
  \end{array}
$$

In real life, of course, we do not know what $\underline{\mathbf{x}}$ is, so we cannot calculate these numbers.  However, we can use this strategy if we test the equation solver on equations with known solutions.  Such equations are easily constructed.

Start with any non-singular $2 \times 2$ matrix $A$ and vector $\underline{\mathbf{x}}$ and define $\underline{\mathbf{b}}$ by $\underline{\mathbf{b}}=A\underline{\mathbf{x}}$ – remember to use `@` for matrix multiplication.  Now pretend you don't know what $\underline{\mathbf{x}}$ is.  Use the `np.linalg.solve` function to find Numpy's solution for $\underline{\mathbf{x}}$ using $A$ and $\underline{\mathbf{b}}$.

**Hint:** Store the calculated solution in `x1` so you don't overwrite `x`.

Calculate the absolute and relative errors compared to the exact solution $\underline{\mathbf{x}}$ (remember to use the infinity norm).  Are they zero? What does this tell you?

Another way to check the accuracy of a supposed solution is to see how well $\underline{\mathbf{x}}_1$ fits the given system.  To do that we calculate $\underline{\mathbf{b}}_1 = A\underline{\mathbf{x}}_1$ and see if $\underline{\mathbf{b}}_1$ is close to $\underline{\mathbf{b}}$.  The difference $\underline{\mathbf{b}}-\underline{\mathbf{b}}_1$ is called the **residual vector**.  So we might hope that if the residual vector is small, then $\underline{\mathbf{x}}_1$ is a good approximation to the true solution $\underline{\mathbf{x}}$.  We measure the size of the residual vector by looking at the one of the following quantities:

$$
  \begin{array}{lll} 
    & \left\|\underline{\mathbf{b}}-\underline{\mathbf{b}}_1\right\| & \text{(the absolute residual)}, \\
    \text{or } & \left\|\underline{\mathbf{b}}-\underline{\mathbf{b}}_1\right\|/\left\|\underline{\mathbf{b}}\right\| & \text{(the relative residual)}.
  \end{array}
$$

Calculate the absolute and relative residuals.  Are they zero?

You will use both these checking methods, along with the condition number, in the following exercise.  Numpy has a built-in function `np.linalg.cond` which gives the **condition number** of a matrix.

Write a function `errors` that calculates Numpy's solution (using `np.linalg.solve`) and works out the relative error and relative residual for a given matrix $A$ and known solution vector $\underline{\mathbf{x}}$.  `errors` should take a matrix `A` and solution vector `x` as parameters, and return the relative error, relative residual, and condition number:
```
return rel_error, rel_resid, cond
```
The multiple outputs (a tuple) can be assigned individual names when you call the function like this:
```
rel_error, rel_resid, cond = errors(A, x)
```

In [ ]:
type your code here

### Hilbert matrices

Hilbert matrices are a badly conditioned type of matrix (that arise if you try to find least squares polynomial fits without using orthogonal polynomials).  Scipy has a function which sets up $n \times n$ Hilbert matrices.  Run `scipy.linalg.hilbert?` to see how to get a Hilbert matrix.

Define $A$ to be the $5 \times 5$ Hilbert matrix, and $\underline{\mathbf{x}}$ to be a $5 \times 1$ vector of 1's.

Run `errors` to find the relative error, relative residual, and condition number.

In [ ]:
type your code here

Now let's investigate how the errors, residuals and condition number behave for different sized Hilbert matrices.  Below, write a function `hilbert_errors` that runs `errors` so that it calculates the relative error, the relative residual and the condition number for a series of Hilbert matrices of size $n = 2, 3, \ldots$ up to size $n = 15$.

Your function should store the results for each matrix as a row of numbers, `results = []` then `results.append(...)` (as you did in Lab 2), which should contain: the size of the matrix ($n$), the relative error, relative residual and condition number.

**Hint:** You can make a list containing the elements of a tuple using `*` as we did above, e.g. `[n, *errors(...)]`.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 2</h3>

  <ul>
    <li>What is the first value of $n$ for which the calculated solution is no longer accurate to 4 significant figures?</li>
    <li>Are the residual vectors a good guide to the accuracy of the calculated solution?</li>
    <li>Is the condition number a good guide to the accuracy of the calculated solution?</li>
  </ul>
</div>

In [ ]:
type your code here

Now, back to multidimensional Newton's method.

## Numerical approximation of the Jacobian

We have already made a multimensional Newton's method that will work for any function (hopefully it will cope with a 3D system like the one below).  However, it requires a manually derived Jacobian function, which can be a lot of work.  We can instead make our function estimate the Jacobian numerically.  Copy your `multinewt` function into the cell below, and modify it so that it has a default `jacobian` parameter value of `None`, and to have an `h` parameter, like this:
```
def multinewt(x, num_iter, func, jacobian=None, tol=1e-4, h=1e-6):
```
Make your new function use a Jacobian function if one is passed in, and otherwise estimate the Jacobian matrix as follows.  The formula for calculating the partial derivative $\partial f_i/\partial x_j$ numerically is really the same as the formula for ordinary derivatives (which you used at the end of Lab 2):

$$
  \frac{\partial f_i}{\partial x_j}
    \approx \frac{f_i(x_1,\ldots,x_j+h,\ldots,x_n )-f_i(x_1,\ldots,x_j,\ldots,x_n)}{h}.
$$

This looks like you might need two two loops, go cover every element of the Jacobian matrix (and that will work), but actually you can do it a whole column at a time, since $f$ produces a vector output.  To do it column-by-column, you will need to evaluate $f$ at the current $x_k$ with $h$ added to one of the elements.  The easiest way to do that is to make a copy of `x` and modify it:
```
xh = x.copy()
xh[i] += h
```

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 3</h3>

  Use your function to find the solution to the system
  \begin{align*}
    x_1^2 - x_1 x_2 - 10 &= 0 \\
    x_2 + 3x_1 x_2^2 - 57 &= 0 \\
    x_1 + x_2 - 2x_3 &= 0
  \end{align*}
  using an initial approximation of $\underline{\mathbf{x}} = (1.5, 3.5, 2.5)$.  Check that your answer gives a small residual.
</div>

In [ ]:
type your code here